# Weights & Biases Walkthrough

# Preface

The goal of this tutorial is to show steps in Gradient to use the Weights & Biases Python library to track model runs and model parameters, create artifacts, and compare models to determine best performing model. 

The context of this tutorial will be to try test and track multiple values for learning rate and epochs for a ResNet model built in PyTorch.


# Installation and Setup

In [ ]:
!pip install wandb -q

In [ ]:
import wandb
import os

os.environ["WANDB_NOTEBOOK_NAME"] = "./train_model_wandb.ipynb"

# Login

In [ ]:
wandb.login(key='1305ff3ca47ed8cd6735ba50a3b2f6697ff94916')

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb.login(key='<your-wandb-api-key>')

# Initalizing a Model Run and Logging

In [ ]:
config={
         "epochs": 5,
         "batch_size": 128,
         "lr": 1e-3,
        }

### Train ResNet 18

In [ ]:
import time
import torch.nn as nn
import torch.optim as optim
import torch

from resnet import resnet18, resnet34
from load_data import trainloader, valloader

Files already downloaded and verified


Files already downloaded and verified


In [ ]:
def validate_model(model, valid_dl, loss_func, device):
    
    # Compute performance of the model on the validation dataset
    model.eval()
    val_loss = 0.
    with torch.inference_mode():
        correct = 0
        for i, (images, labels) in enumerate(valid_dl, 0):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            val_loss += loss_func(outputs, labels)*labels.size(0)

            # Compute accuracy and accumulate
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

    return val_loss / len(valid_dl.dataset), correct / len(valid_dl.dataset)

In [ ]:
model_name = 'ResNet18'

with wandb.init(project="test-project", config=config, name=model_name):

    model = resnet18(3, 10)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    step = 0
    epoch_durations = []
    for epoch in range(config['epochs']):
        
        epoch_start_time = time.time()
        batch_checkpoint=50
        running_loss = 0.0
        model.train()

        for i, data in enumerate(trainloader, 0):
        
            inputs, labels = data[0].to(device), data[1].to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward + Backward + Optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
            running_loss += loss.item()
            
            # log every 50 mini-batches
            if i % batch_checkpoint == batch_checkpoint-1:    # log every 50 mini-batches
                step +=1
                print('epoch:', epoch + ((i+1)/len(trainloader)))
                wandb.log({"train_loss": running_loss/batch_checkpoint, "epoch": epoch + ((i+1)/len(trainloader))}, step=step)
            
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / batch_checkpoint))
                    
                running_loss = 0.0

        # Log validation metrics
        val_loss, accuracy = validate_model(model, valloader, criterion, device)
        wandb.log({"val_loss": val_loss, "val_accuracy": accuracy}, step=step)
        print(f"Valid Loss: {val_loss:3f}, accuracy: {accuracy:.2f}")
        
        epoch_duration = time.time() - epoch_start_time
        wandb.log({"epoch_runtime (seconds)": epoch_duration}, step=step)

        epoch_durations.append(epoch_duration)

    avg_epoch_runtime = sum(epoch_durations) / len(epoch_durations)
    wandb.log({"avg epoch runtime (seconds)": avg_epoch_runtime})

print('Training Finished')

epoch: 0.125
[1,    50] loss: 1.855


epoch: 0.25
[1,   100] loss: 1.515


epoch: 0.375
[1,   150] loss: 1.428


epoch: 0.5
[1,   200] loss: 1.317


epoch: 0.625
[1,   250] loss: 1.249


...

epoch: 4.875
[5,   350] loss: 0.541


epoch: 5.0
[5,   400] loss: 0.547


Valid Loss: 1.019831, accuracy: 0.68


avg epoch runtime (seconds),▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch_runtime (seconds),▁█▃▁▄
train_loss,█▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▂▂▂
val_accuracy,▁▆▆██
val_loss,█▂▂▁▂
avg epoch runtime (seconds),4.95499
epoch,5.0
epoch_runtime (seconds),4.98119
train_loss,0.54706
val_accuracy,0.6765


Training Finished


### Train ResNet34

In [ ]:
model_name = 'ResNet34'

with wandb.init(project="test-project", config=config, name=model_name):

    model = resnet34(3, 10)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    step = 0
    epoch_durations = []
    for epoch in range(config['epochs']):
        
        epoch_start_time = time.time()
        batch_checkpoint=50
        running_loss = 0.0
        model.train()

        for i, data in enumerate(trainloader, 0):
        
            inputs, labels = data[0].to(device), data[1].to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward + Backward + Optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
            running_loss += loss.item()
            
            # log every 50 mini-batches
            if i % batch_checkpoint == batch_checkpoint-1:    # log every 50 mini-batches
                step +=1
                print('epoch:', epoch + ((i+1)/len(trainloader)))
                wandb.log({"train_loss": running_loss/batch_checkpoint, "epoch": epoch + ((i+1)/len(trainloader))}, step=step)
            
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / batch_checkpoint))
                    
                running_loss = 0.0

        # Log validation metrics
        val_loss, accuracy = validate_model(model, valloader, criterion, device)
        wandb.log({"val_loss": val_loss, "val_accuracy": accuracy}, step=step)
        print(f"Valid Loss: {val_loss:3f}, accuracy: {accuracy:.2f}")
        
        epoch_duration = time.time() - epoch_start_time
        wandb.log({"epoch_runtime (seconds)": epoch_duration}, step=step)

        epoch_durations.append(epoch_duration)

    avg_epoch_runtime = sum(epoch_durations) / len(epoch_durations)
    wandb.log({"avg epoch runtime (seconds)": avg_epoch_runtime})

print('Training Finished')

epoch: 0.125
[1,    50] loss: 1.997


epoch: 0.25
[1,   100] loss: 1.594


epoch: 0.375
[1,   150] loss: 1.452


epoch: 0.5
[1,   200] loss: 1.342


epoch: 0.625
[1,   250] loss: 1.284


...

epoch: 2.75
[3,   300] loss: 0.746


epoch: 2.875
[3,   350] loss: 0.743


epoch: 3.0
[3,   400] loss: 0.742


epoch,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇██
epoch_runtime (seconds),▁▃█
train_loss,█▆▅▄▄▄▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
epoch,3.0
epoch_runtime (seconds),23.74247
train_loss,0.74153


Training Finished


# Artifacts

In [ ]:
from load_data import testloader

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

with wandb.init(project='test-project'):
    artifact = wandb.Artifact('cifar10_image_predictions', type='predictions')

    columns=['image', 'label', 'prediction']
    for digit in range(10):
        columns.append("score_" + classes[digit])

    pred_table = wandb.Table(columns=columns)   

    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            for j, image in enumerate(inputs,0):
                pred_table.add_data(wandb.Image(image), classes[labels[j].item()], classes[predicted[j]], *outputs[j])
                break

    artifact.add(pred_table, "cifar10_predictions")
    wandb.log_artifact(artifact)

# Sweeps

In [ ]:
sweep_config = {
                'method': 'grid',
                'metric': {'goal': 'minimize', 'name': 'val_loss'},
                'parameters': {
                    'batch_size': {'values': [16, 128]},
                    'epochs': {'values': [6]},
                    'lr': {'values': [1e-3, 1e-4]},
                    'model': {'values': ['ResNet18', 'ResNet34']}
                    }
}

In [ ]:

def train(config = None):
    with wandb.init(project='test-project', config=config):
        config = wandb.config

        if config['model']=='ResNet18':
            model = resnet18(3,10)
        else:
            model = resnet34(3,10)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9)

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model.to(device)

        step = 0
        epoch_durations = []
        for epoch in range(config['epochs']):
            
            epoch_start_time = time.time()
            batch_checkpoint=50
            running_loss = 0.0
            model.train()

            for i, data in enumerate(trainloader, 0):
            
                inputs, labels = data[0].to(device), data[1].to(device)
                
                # Zero the parameter gradients
                optimizer.zero_grad()
                
                # Forward + Backward + Optimize
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            
                running_loss += loss.item()
                
                # log every 50 mini-batches
                if i % batch_checkpoint == batch_checkpoint-1:    # log every 50 mini-batches
                    step +=1
                    print('epoch:', epoch + ((i+1)/len(trainloader)))
                    wandb.log({"train_loss": running_loss/batch_checkpoint, "epoch": epoch + ((i+1)/len(trainloader))}, step=step)
                
                    print('[%d, %5d] loss: %.3f' %
                        (epoch + 1, i + 1, running_loss / batch_checkpoint))
                        
                    running_loss = 0.0

            # Log validation metrics
            val_loss, accuracy = validate_model(model, valloader, criterion, device)
            wandb.log({"val_loss": val_loss, "val_accuracy": accuracy}, step=step)
            print(f"Valid Loss: {val_loss:3f}, accuracy: {accuracy:.2f}")
            
            epoch_duration = time.time() - epoch_start_time
            wandb.log({"epoch_runtime (seconds)": epoch_duration}, step=step)

            epoch_durations.append(epoch_duration)

        avg_epoch_runtime = sum(epoch_durations) / len(epoch_durations)
        wandb.log({"avg epoch runtime (seconds)": avg_epoch_runtime})

    print('Training Finished')

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="test-project")
wandb.agent(sweep_id, function=train)

Create sweep with ID: wuz0x8de
Sweep URL: https://wandb.ai/joshuapaperspace/test-project/sweeps/wuz0x8de


wandb: Agent Starting Run: mryjmedd with config:


wandb: 	batch_size: 16


wandb: 	epochs: 6


wandb: 	lr: 0.001


...

epoch: 4.625
[5,   250] loss: 0.795


epoch: 4.75
[5,   300] loss: 0.780


epoch: 4.875
[5,   350] loss: 0.790


epoch: 5.0
[5,   400] loss: 0.761


Valid Loss: 1.151576, accuracy: 0.59


In [ ]:
hi = 'ResNet34-epochs-10'

In [ ]:
pip install wandb -qqq
python train_model_wandb.py --layers=34 --epochs=10

In [ ]:
import wandb
import os

In [ ]:
os.environ["WANDB_NOTEBOOK_NAME"] = "./train_model_wandb.ipynb"

# Log in to your W&B account
## Access secrets through SDK?
wandb.login(key='1305ff3ca47ed8cd6735ba50a3b2f6697ff94916')

wandb: Currently logged in as: joshuapaperspace (use `wandb login --relogin` to force relogin)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import sys, getopt
import torch.optim as optim
import numpy as np
import torch
import torch.nn as nn
import time

from resnet import resnet18, resnet34
from load_data import trainloader, testloader, valloader
from config.config import model_config


# wandb.init(project="resnet-test")

models = ['ResNet18', 'ResNet34']

for model in models:
    with wandb.init(project="resnet-test", config=model_config, name=model):

        print(model_config['epochs'])
        print(model_config['batch_size'])
        print(model_config['lr'])
        print(model)

        if model == 'ResNet18':
            model = resnet18(3, 10)
        else:
            model = resnet34(3, 10)

        criterion = nn.CrossEntropyLoss()
        # optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
        optimizer = optim.SGD(model.parameters(), lr=model_config['lr'], momentum=0.9)

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model.to(device)

        step = 0
        epoch_durations = []
        for epoch in range(model_config['epochs']):
            
            start_epoch_time = time.time()
            # print("--- %s seconds ---" % (time.time() - start_time))

            print('epoch:', epoch+1)
            mini_batch_check=50
            running_loss = 0.0
            model.train()

            for i, data in enumerate(trainloader, 0):
                # print(i)

                inputs, labels = data[0].to(device), data[1].to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                # print statistics
                running_loss += loss.item()
                
                if i % mini_batch_check == mini_batch_check-1:    # print every 50 mini-batches
                    step +=1
                    print('inter-epoch:', epoch + ((i+1)/len(trainloader)))
                    wandb.log({"train_loss": running_loss/mini_batch_check, "epoch": epoch + ((i+1)/len(trainloader))}, step=step)

                    print('[%d, %5d] loss: %.3f' %
                        (epoch + 1, i + 1, running_loss / mini_batch_check))
                    
                    running_loss = 0.0
            
            val_loss, accuracy = validate_model(model, valloader, criterion)
                
            # Log validation metrics
            wandb.log({"val_loss": val_loss, "val_accuracy": accuracy}, step=step)
            print(f"Valid Loss: {val_loss:3f}, accuracy: {accuracy:.2f}")
            epoch_duration = time.time() - start_epoch_time
            wandb.log({"epoch_runtime (seconds)": epoch_duration}, step=step)

            epoch_durations.append(epoch_duration)

        avg_epoch_runtime = sum(epoch_durations) / len(epoch_durations)
        wandb.log({"avg epoch runtime (seconds)": avg_epoch_runtime})
        # wandb.finish()
print('Finished Training')

20
125
0.001
ResNet18
epoch: 1


inter-epoch: 0.125
[1,    50] loss: 1.864


inter-epoch: 0.25
[1,   100] loss: 1.498


inter-epoch: 0.375
[1,   150] loss: 1.369


inter-epoch: 0.5
[1,   200] loss: 1.305


...

inter-epoch: 19.875
[20,   350] loss: 0.037


inter-epoch: 20.0
[20,   400] loss: 0.052


Valid Loss: 1.638053, accuracy: 0.72


avg epoch runtime (seconds),▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch_runtime (seconds),▃▄▃▅▅▅▄▇▅▃█▇▃▇▅▄▃▆▁▄
train_loss,█▆▅▅▄▄▃▄▃▃▂▃▂▂▂▂▁▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▆▆▆▇▇▇▇▇█▇▇█▇▇████
val_loss,▃▂▁▁▁▂▃▂▃▅▄▆▆▆▆▇▇▇██
avg epoch runtime (seconds),7.63705
epoch,20.0
epoch_runtime (seconds),7.6177
train_loss,0.05237
val_accuracy,0.717


Finished Training


In [ ]:
def validate_model(model, valid_dl, loss_func, log_images=False, batch_idx=0):
    "Compute performance of the model on the validation dataset and log a wandb.Table"
    model.eval()
    val_loss = 0.
    with torch.inference_mode():
        correct = 0
        for i, (images, labels) in enumerate(valid_dl, 0):
        # for i, (images, labels) in enumerate(valid_dl), leave=False):
            images, labels = images.to(device), labels.to(device)

            # Forward pass ➡
            outputs = model(images)
            val_loss += loss_func(outputs, labels)*labels.size(0)

            # Compute accuracy and accumulate
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

            # Log one batch of images to the dashboard, always same batch_idx.
            # if i==batch_idx and log_images:
                # log_image_table(images, predicted, labels, outputs.softmax(dim=1))
    return val_loss / len(valid_dl.dataset), correct / len(valid_dl.dataset)

In [ ]:
for i, data in enumerate(trainloader, 0):
    print(data)
    exit()

NameError: name 'trainloader' is not defined

In [ ]:
len(trainloader)

12500

In [ ]:
from load_data import trainloader

Files already downloaded and verified


Files already downloaded and verified


In [ ]:
32*391

12512